In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np
try:
    import mxnet as mx
except:
    sys.stderr.write("Cannot import mxnet.\n")
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.exposure import rescale_intensity
from skimage.transform import rotate

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *
from distributed_utilities import *
from visualization_utilities import *

/home/yuncong/Brain/utilities/utilities2015.py:2: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'module://ipykernel.pylab.backend_inline' by the following code:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python2.7/dist-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/local/lib/python2.7/dist-packages

Setting environment for Precision WorkStation


No vtk
No vtkNot using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
Not using image_cache.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
from sklearn.externals import joblib

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.ensemble import GradientBoostingClassifier 

sys.path.append('/home/yuncong/csd395/xgboost/python-package')
try:
    from xgboost.sklearn import XGBClassifier
except:
    sys.stderr.write('xgboost is not loaded.')

In [3]:
batch_size = 256
model_dir_name = 'inception-bn-blue'
model_name = 'inception-bn-blue'
model, mean_img = load_mxnet_model(model_dir_name=model_dir_name, model_name=model_name, 
                                   num_gpus=1, batch_size=batch_size)

/usr/local/lib/python2.7/dist-packages/mxnet/module/base_module.py:53: UserWarning: You created Module with Module(..., label_names=['softmax_label']) but input with name 'softmax_label' is not found in symbol.list_arguments(). Did you mean one of:
	data
  warnings.warn(msg)
/usr/local/lib/python2.7/dist-packages/mxnet/module/base_module.py:65: UserWarning: Data provided by label_shapes don't match names specified by label_names ([] vs. ['softmax_label'])
  warnings.warn(msg)


In [4]:
win_id = 7

In [5]:
stacks = ['MD589', 'MD585', 'MD594']
# stacks = ['MD661', 'MD662']
# stacks = ['MD661', 'ChatCryoJane201710']
# stacks = ['MD642', 'MD661']
# stacks = ['MD661', 'MD662', 'ChatCryoJane201710']
# stacks = ['MD589', 'MD661']
# stacks = ['MD589', 'ChatCryoJane201710']

In [10]:
grid_indices_lookup_allStacks = {}

for stack in stacks:
    try:
#         grid_indices_lookup_allStacks[stack] = \
#         DataManager.load_annotation_to_grid_indices_lookup(stack=stack, win_id=win_id,
#                                                            by_human=False, timestamp='latest',
#                                                            detector_id_f=1,
#                                                           return_locations=True)            

        grid_indices_lookup_allStacks[stack] = \
        DataManager.load_annotation_to_grid_indices_lookup(stack=stack, win_id=win_id,
                                                           by_human=True, timestamp='latest',
                                                          return_locations=True)            

    except Exception as e:
        print e
        sys.stderr.write("Fail to load annotation grid lookup for %s.\n" % stack)

aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD589" "/home/yuncong/CSHL_labelings_v3/MD589" --exclude "*" --include "*win7*grid_indices_lookup*"
latest timestamp:  10042017100807
aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD585" "/home/yuncong/CSHL_labelings_v3/MD585" --exclude "*" --include "*win7*grid_indices_lookup*"
latest timestamp:  08012017212649
aws s3 cp --recursive "s3://mousebrainatlas-data/CSHL_labelings_v3/MD594" "/home/yuncong/CSHL_labelings_v3/MD594" --exclude "*" --include "*win7*grid_indices_lookup*"
latest timestamp:  07302017183604


In [11]:
from itertools import chain
all_labels = sorted(list(set(chain.from_iterable(set(grid_indices_lookup_allStacks[st].columns.tolist()) 
                                                 for st in stacks))))

In [13]:
# train_stacks = ['MD661', 'MD662']
# train_stack_stain = {'MD661': 'F', 'MD662': 'F'}

train_stacks = ['MD585', 'MD589', 'MD594']
train_stack_stain = {'MD585': 'N', 'MD589': 'N', 'MD594': 'N'}

In [14]:
train_stack_section_number = defaultdict(dict)

for st in train_stacks:
    for name_u in all_known_structures:
#         if name_u == '4N' or name_u == '10N':
#             train_stack_section_number[st][name_u] = 20
#         else:
#             train_stack_section_number[st][name_u] = 10
        train_stack_section_number[st][name_u] = 99

In [15]:
classifier_id = 998

In [16]:
for structure in all_known_structures:
# for structure in [
#  'VCA',
#  'VCP',
#  'DC',
#  'AP',
#  '12N',
#  'RtTg',
#  'SC',
#  'IC']:
# for structure in ['LRt']:
    
    surround_margins = [200]

# structure = 'Amb'
#     structure = '7N'
    # structure = 'SNR'
    positive_labels = [structure]
    negative_labels = get_negative_labels(structure, 'neg_has_all_surround', margin_um=200, 
                                          labels_found=all_labels)    
    
    positive_addresses_traindata_all_stacks = {}
    negative_addresses_traindata_all_stacks = {}

    for train_stack in train_stacks:
        
        train_sections = list(chain(*[grid_indices_lookup_allStacks[train_stack][pl].dropna(how='any').index.tolist() 
                                      for pl in positive_labels]))

        ###################################################################################

        n_train_sections = train_stack_section_number[train_stack][structure]
        # n_train_sections = 10
        # n_train_sections = len(train_sections)

        if train_stack_stain[train_stack] == 'F':
            neurotrace_sections = []
            nissl_sections = []
            for sec in train_sections:
                if metadata_cache['sections_to_filenames'][train_stack][sec].split('-')[1][0] == 'F':
                    neurotrace_sections.append(sec)
                else:
                    nissl_sections.append(sec)
            random_train_sections = np.random.choice(neurotrace_sections, min(len(neurotrace_sections), n_train_sections), replace=False)
        else:
            random_train_sections = np.random.choice(train_sections, min(len(train_sections), n_train_sections), replace=False)

        positive_addresses_traindata_all_stacks[train_stack] = sorted([(train_stack, sec, tuple(loc))
for nl in set(positive_labels) & set(grid_indices_lookup_allStacks[train_stack].columns)
  for sec, locs in grid_indices_lookup_allStacks[train_stack][nl].loc[random_train_sections].dropna().iteritems()
  for loc in locs])
        
        negative_addresses_traindata_all_stacks[train_stack] = sorted([(train_stack, sec, tuple(loc))
for nl in set(negative_labels) & set(grid_indices_lookup_allStacks[train_stack].columns)
  for sec, locs in grid_indices_lookup_allStacks[train_stack][nl].loc[random_train_sections].dropna().iteritems()
  for loc in locs])
   
    positive_addresses_traindata = sum(positive_addresses_traindata_all_stacks.values(), [])
    negative_addresses_traindata = sum(negative_addresses_traindata_all_stacks.values(), [])

    print '# positive train =', len(positive_addresses_traindata)
    print '# negative train =', len(negative_addresses_traindata)

    del positive_addresses_traindata_all_stacks, negative_addresses_traindata_all_stacks
    
    # features_dict = {(scheme, tfv): {} for scheme in schemes for tfv in transforms}
    features_dict = defaultdict(dict)
    
    # train_scheme = 'stretch_min_max'
    # train_scheme = 'normalize_mu_sigma_global_(-1,5)'
    train_scheme = 'normalize_mu_region_sigma_wholeImage_(-1,5)'
    # train_scheme = 'normalize_mu_region_sigma_wholeImage_(-1,9)'
    # train_scheme = 'none'
    addresses_to_compute = positive_addresses_traindata + negative_addresses_traindata
    
    features_loaded = read_cnn_features(addresses=addresses_to_compute, 
                                        scheme=train_scheme, win_id=7, prep_id=2, 
                                        model=model, mean_img=mean_img, model_name=model_name, 
                                        batch_size=batch_size)
        
    for addr, f in izip(addresses_to_compute, features_loaded):
        features_dict[(train_scheme, 'transform0')][addr] = f
    
#     for i, f in enumerate(features_loaded):
#         features_dict[(train_scheme, 'transform0')][addresses_to_compute[i]] = f

    ###########################################################
    
    # n_train_list = [10, 100, 200, 500, 1000, 2000, 5000, 10000, 15000]
#     n_train_list = [10, 1000]
    n_train_list = [15000]
#     test_metrics_all_ntrain = defaultdict(lambda: defaultdict(list))
    train_metrics_all_ntrain = defaultdict(lambda: defaultdict(list))

    for n_train in n_train_list:

        for trial in range(1):

            # If train and test data are from different sets
        #     n_train_pos = 5000
            n_train_pos = min(n_train, len(positive_addresses_traindata))
#             if len(positive_addresses_traindata) < n_train_pos:
#                 continue
            training_pos_indices = np.random.choice(range(len(positive_addresses_traindata)), n_train_pos, replace=False)
            
#             n_test_pos = 1000
#             test_pos_indices = np.random.choice(range(len(positive_addresses_testdata)),
#                                                 size=min(len(positive_addresses_testdata), n_test_pos), 
#                                                 replace=False)

            # If train and test are from same set
        #     n_pos_total = len(positive_addresses)
        #     n_train_pos = 1000
        #     training_pos_indices = np.random.choice(range(n_pos_total), n_train_pos, replace=False)
        #     test_pos_indices = np.random.choice(np.setdiff1d(range(n_pos_total), training_pos_indices),
        #                                         size=min(2000, n_pos_total-n_train_pos), replace=False)
        #     n_test_pos = len(test_pos_indices)

            ###############

            # If train and test data are from different sets
        #     n_train_neg = 5000
            n_train_neg = n_train_pos
            training_neg_indices = np.random.choice(range(len(negative_addresses_traindata)), n_train_neg, replace=False)
#             n_test_neg = 1000
#             test_neg_indices = np.random.choice(range(len(negative_addresses_testdata)),
#                                                 size=min(len(negative_addresses_testdata), n_test_neg), 
#                                                 replace=False)

            # If train and test are from same set
        #     n_neg_total = len(negative_addresses)
        #     n_train_neg = 1000
        #     training_neg_indices = np.random.choice(range(n_neg_total), n_train_neg, replace=False)
        #     test_neg_indices = np.random.choice(np.setdiff1d(range(n_neg_total), training_neg_indices), 
        #                                         size=min(2000, n_pos_total-n_train_pos), replace=False)
        #     n_test_neg = len(test_neg_indices)

            print "Training: %d positive, %d negative" % (n_train_pos, n_train_neg)
#             print "Test: %d positive, %d negative" % (n_test_pos, n_test_neg)

            ################

            # If train and test data are from different sets
            addresses_train_pos = [positive_addresses_traindata[i] for i in training_pos_indices]
            addresses_train_neg = [negative_addresses_traindata[i] for i in training_neg_indices]

            #################

#             for augment_training in [True, False]:
            for augment_training in [False]:

                feature_classifier_alg = 'lr'
        #             feature_classifier_alg = 'xgb2'
        #             feature_classifier_alg = 'lin_svc'
        #             feature_classifier_alg = 'lin_svc_calib'
                sample_weights = None   

                if augment_training:
                    train_transforms = range(8)
                else:
                    train_transforms = range(1)

                test_transforms = range(1)

                features_train_pos = {(train_scheme, 'transform%d'%tf_variant): 
                                      [features_dict[(train_scheme, 'transform%d'%tf_variant)][addr] 
                                       for addr in addresses_train_pos]
                                          for tf_variant in train_transforms}
                features_train_neg = {(train_scheme, 'transform%d'%tf_variant): 
                                      [features_dict[(train_scheme, 'transform%d'%tf_variant)][addr] 
                                       for addr in addresses_train_neg]
                                          for tf_variant in train_transforms}

                if augment_training:

                    train_data_list = []
                    train_label_list = []
                    for tf in range(8):
                        train_data = np.r_[features_train_pos[(train_scheme,'transform%d'%tf)], 
                                           features_train_neg[(train_scheme,'transform%d'%tf)]]
                        train_data_list.append(train_data)

                        train_labels = np.r_[np.ones((len(features_train_pos[(train_scheme,'transform%d'%tf)]), )), 
                                             -np.ones((len(features_train_neg[(train_scheme,'transform%d'%tf)]), ))]
                        train_label_list.append(train_labels)

                    train_data = np.concatenate(train_data_list)
                    train_labels = np.concatenate(train_label_list)        
                else:

                    train_data = np.r_[features_train_pos[(train_scheme,'transform0')], 
                                       features_train_neg[(train_scheme,'transform0')]]
                    train_labels = np.r_[np.ones((len(features_train_pos[(train_scheme,'transform0')]), )), 
                                         -np.ones((len(features_train_neg[(train_scheme,'transform0')]), ))]


                clf = train_binary_classifier(train_data, train_labels,
                                       alg=feature_classifier_alg, 
                                       sample_weights=sample_weights)
                
                del train_data, features_train_pos, features_train_neg, features_loaded, features_dict

                clf_fp = DataManager.get_classifier_filepath(classifier_id=classifier_id, structure=structure)
                create_parent_dir_if_not_exists(clf_fp)
                joblib.dump(clf, clf_fp)
                upload_to_s3(clf_fp)

# positive train = 25288
# negative train = 65035
('MD594', 163)
rm -rf "/media/yuncong/BstemAtlasData/CSHL_patch_features/inception-bn-blue/MD594/MD594_prep2_normalize_mu_region_sigma_wholeImage_(-1,5)_win7/MD594-N33-2015.08.26-22.44.00_MD594_1_0097_prep2_normalize_mu_region_sigma_wholeImage_(-1,5)_win7_inception-bn-blue_features.bp" && mkdir -p "/media/yuncong/BstemAtlasData/CSHL_patch_features/inception-bn-blue/MD594/MD594_prep2_normalize_mu_region_sigma_wholeImage_(-1,5)_win7"
aws s3 cp "s3://mousebrainatlas-data/CSHL_patch_features/inception-bn-blue/MD594/MD594_prep2_normalize_mu_region_sigma_wholeImage_(-1,5)_win7/MD594-N33-2015.08.26-22.44.00_MD594_1_0097_prep2_normalize_mu_region_sigma_wholeImage_(-1,5)_win7_inception-bn-blue_features.bp" "/media/yuncong/BstemAtlasData/CSHL_patch_features/inception-bn-blue/MD594/MD594_prep2_normalize_mu_region_sigma_wholeImage_(-1,5)_win7/MD594-N33-2015.08.26-22.44.00_MD594_1_0097_prep2_normalize_mu_region_sigma_wholeImage_(-1,5)_win7_inception

IOError: [Errno 2] No such file or directory: '/media/yuncong/BstemAtlasData/CSHL_patch_features/inception-bn-blue/MD594/MD594_prep2_normalize_mu_region_sigma_wholeImage_(-1,5)_win7/MD594-N33-2015.08.26-22.44.00_MD594_1_0097_prep2_normalize_mu_region_sigma_wholeImage_(-1,5)_win7_inception-bn-blue_features.bp'

- classfier 999 is trained from 15,000 pos and 15,000 neg from both MD661 and MD662
- classfier 998 is trained from 15,000 pos and 15,000 neg from MD589/MD585/MD594